In [2]:
import os
import time
import tensorflow as tf
# modified to solve placeholder() error
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
import numpy as np
from glob import glob
import datetime
import random
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
from  tensorflow import keras
from  tensorflow.keras import layers
import base64

In [3]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


2024-10-23 14:18:47.133352: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-10-23 14:18:47.133392: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-10-23 14:18:47.133402: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-10-23 14:18:47.133680: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-23 14:18:47.133709: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-10-23 14:18:47.138576: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been bui

In [4]:
image_size = (256,256)
batch_size = 8

train_ds = keras.preprocessing.image_dataset_from_directory(
    "nice_smaller",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = keras.preprocessing.image_dataset_from_directory(
    "nice_smaller",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 1036 files belonging to 2 classes.
Using 829 files for training.
Found 1036 files belonging to 2 classes.
Using 207 files for validation.


2024-10-23 14:18:48.993965: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-23 14:18:48.993987: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)

In [6]:
augmented_train_ds = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))

In [7]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [8]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

In [9]:
model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [11]:
epochs = 80

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, validation_data=val_ds,
)

Epoch 1/80
104/104 ━━━━━━━━━━━━━━━━━━━━ 41s 320ms/step - accuracy: 0.9416 - loss: 0.1650 - val_accuracy: 0.8599 - val_loss: 0.3215
Epoch 2/80
104/104 ━━━━━━━━━━━━━━━━━━━━ 31s 301ms/step - accuracy: 0.9474 - loss: 0.1435 - val_accuracy: 0.8647 - val_loss: 0.3480
Epoch 3/80
104/104 ━━━━━━━━━━━━━━━━━━━━ 30s 288ms/step - accuracy: 0.9403 - loss: 0.1473 - val_accuracy: 0.9469 - val_loss: 0.1457
Epoch 4/80
104/104 ━━━━━━━━━━━━━━━━━━━━ 30s 286ms/step - accuracy: 0.9579 - loss: 0.1177 - val_accuracy: 0.8696 - val_loss: 0.3497
Epoch 5/80
104/104 ━━━━━━━━━━━━━━━━━━━━ 30s 285ms/step - accuracy: 0.9527 - loss: 0.1274 - val_accuracy: 0.8937 - val_loss: 0.3130
Epoch 6/80
104/104 ━━━━━━━━━━━━━━━━━━━━ 30s 289ms/step - accuracy: 0.9322 - loss: 0.1687 - val_accuracy: 0.9082 - val_loss: 0.2617
Epoch 7/80
104/104 ━━━━━━━━━━━━━━━━━━━━ 30s 284ms/step - accuracy: 0.9481 - loss: 0.1448 - val_accuracy: 0.9469 - val_loss: 0.1271
Epoch 8/80
104/104 ━━━━━━━━━━━━━━━━━━━━ 31s 295ms/step - accuracy: 0.9458 - loss: 0

In [ ]:
f = open("imagelst.txt", "r")
for fnam in f:
    fnam = fnam.replace("\n", "")
    img = keras.preprocessing.image.load_img( fnam, target_size=image_size )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    predictions = model.predict(img_array)
    score = predictions[0]
    print( fnam , " , This image is %.2f percent female and %.2f percent male."
        % (100 * (1 - score), 100 * score) )
f.close()